In [6]:
import spacy

In [13]:
text = "lihat kontrak yg di bawah 20 tahun"

nlp = spacy.load("output/model-last")
doc = nlp(text)

[(w.text, w.label_) for w in doc.ents]

c:\Users\paoma\miniconda3\lib\site-packages\spacy_transformers\layers\hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self._model.load_state_dict(torch.load(fi

[('kontrak', 'SUBJECT'), ('bawah', 'LOGIC'), ('20 tahun', 'VALUE')]